In [1]:
import PyRadioTrace.models as models
from PyRadioTrace.models import GeoModel
from PyRadioTrace.jones_stephenson import Raytracer
import datetime as dt 
import numpy as np 
import matplotlib.pyplot as plt
import scipy.constants as const

# Models

In [2]:
test_lat = 40 # degrees
test_lon = -110 # degrees
test_alt = 40e3 # meters

neutral = models.ScaleHeight(N0 = 400, H = 7e3)
magneto = models.IGRF(dt.datetime(2023, 1, 1))
iono = models.EpsteinLayersModel()

All neutral atmosphere models should return the index of refraction as well the spherical spatial derivatives.

In [3]:
neutral(test_lat, test_lon, test_alt)

(np.float64(1.0000013194023023),
 array([-1.88486043e-10,  0.00000000e+00,  0.00000000e+00]))

All ionosphere models should return the electron density as well the spherical spatial derivatives.

In [4]:
iono(test_lat, test_lon, test_alt)

(21935836.509093076, array([935.39927486,   0.        ,   0.        ]))

The IGRF object returns the magnetic field vector, magnetic field jacobian, and magnetic field strength spherical derivatives. 

In [5]:
magneto(test_lat, test_lon, test_alt)

(array([[-3.57350256e+98],
        [ 8.91385642e+97],
        [-9.57329235e+97]]),
 array([[[ 3.57350256e+95],
         [-2.16218843e+97],
         [-2.18299095e+96]],
 
        [[-8.91385642e+94],
         [-1.29969497e+98],
         [-8.10884744e+96]],
 
        [[ 9.57329235e+94],
         [ 1.04755516e+98],
         [-3.01328857e+96]]]),
 array([[3.80538673e+95],
        [1.68324966e+98],
        [8.92181396e+96]]))

# Raytracing 

Instantiate a Raytracer object with the models above 

In [6]:
Raytrace  = Raytracer(iono = iono, magneto = None, neutral = neutral)

In [7]:
rtol = 1e-10
atol = 1e-8
transmit_lat = 0
transmit_lon = 0.5
transmit_alt = 800e3
f = 1575.42e6

az = 90
el = -10

In [8]:
distances_to_evaluate = np.arange(0, 100e3, 1)

In [9]:
ray_solution = Raytrace.ray_propagate(transmit_lat,
                                    transmit_lon, 
                                    transmit_alt,
                                    az,
                                    el,
                                    f, 
                                    group_path_distances = distances_to_evaluate,
                                    rtol = rtol,
                                    atol = atol)
ray_solution

  message: The solver successfully reached the end of the integration interval.
  success: True
   status: 0
        t: [ 0.000e+00  1.000e+00 ...  1.000e+05  1.000e+05]
        y: [[ 7.178e+06  7.178e+06 ...  7.161e+06  7.161e+06]
            [ 1.571e+00  1.571e+00 ...  1.571e+00  1.571e+00]
            ...
            [ 0.000e+00 -1.000e+00 ... -1.000e+05 -1.000e+05]
            [ 0.000e+00  1.000e+00 ...  1.000e+05  1.000e+05]]
      sol: None
 t_events: None
 y_events: None
     nfev: 56
     njev: 0
      nlu: 0

Convert solution to ECEF or geodetic coordinates

In [10]:
path_x, path_y, path_z = GeoModel.convert_spherical_to_ecef(ray_solution.y[0,:], ray_solution.y[1,:], ray_solution.y[2,:])
path_lats_both, path_lons_both, path_alts_both = GeoModel.convert_spherical_to_lla(ray_solution.y[0,:], ray_solution.y[1,:], ray_solution.y[2,:])

# Rayhoming

In [11]:
receive_lat = 0
receive_lon = 105
receive_alt = 20200e3


Calculate the straight line az and el 

In [12]:
az, el, range_val = GeoModel.geodetic2aer(transmit_lat, transmit_lon, transmit_alt, receive_lat, receive_lon, receive_alt)

In [20]:
import cProfile
from pstats import SortKey
import pstats
with cProfile.Profile() as pr:
    solution = Raytrace.ray_home_onedim(transmit_lat, transmit_lon, transmit_alt, receive_lat, receive_lon, receive_alt, f, xatol=1e-10, resolution_m = 0.1, rtol = rtol, atol = atol)
    print(solution)
    stats = pstats.Stats(pr)
    stats.sort_stats(SortKey.TIME).print_stats(20)  # Sort by total time

 message: Solution found.
 success: True
  status: 0
     fun: 6.123233995736766e-17
       x: -27.913195772520098
     nit: 26
    nfev: 26
         813588 function calls (813581 primitive calls) in 3.176 seconds

   Ordered by: internal time
   List reduced from 185 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       28    0.825    0.029    1.458    0.052 /home/jason/anaconda3/envs/raytrace/lib/python3.10/site-packages/scipy/integrate/_ivp/rk.py:560(_call_impl)
       28    0.484    0.017    0.484    0.017 {method 'cumprod' of 'numpy.ndarray' objects}
       26    0.444    0.017    0.444    0.017 /home/jason/Documents/research/raytracing/gnssro_raytracer/PyTrace/src/PyRadioTrace/models.py:123(distance_between_two_spherical_vectors)
       52    0.160    0.003    0.161    0.003 /home/jason/anaconda3/envs/raytrace/lib/python3.10/site-packages/numpy/_core/shape_base.py:294(hstack)
       28    0.147    0.005    0.147    0.005 {me

In [14]:
print(solution.x - el)

0.34838912290069146


In [16]:
import numba 
from numba import float64
from numba.experimental import jitclass

spec = [
    ('value', float64),
    ('array', float64[:]),
]

@jitclass(spec)
class TestClass:
    def __init__(self, value):
        self.value = value
        self.array =  np.zeros(value, dtype=np.float64)

    def add(self, x):
        return x + self.value
    def __call__(self, x):
        
        self.array[0] += x*self.value
        return self.array

In [17]:
test_class = TestClass(10)

TypeError: Method '__call__' is not supported.

In [ ]:
test_class.add(5)